In [1]:
!nvidia-smi

Wed Feb  9 04:28:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

In [ ]:
!pip install tensorflow

In [4]:
import gc
import tensorflow as tf
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
gc.collect()

0

In [5]:
import pandas as pd
import numpy as np
from collections import Counter
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
train_df = pd.read_csv('mami_training.csv', sep='\t', lineterminator='\r')
test_df = pd.read_csv('Test.csv', sep='\t', lineterminator='\n')
test_labels = pd.read_csv('test_labels.txt',header=None, sep='\t', lineterminator='\n')
X = train_df['Text Transcription'].copy()
for i in range(len(X)):
  X[i] = X[i].lower()
X_test_model = test_df['Text Transcription'].to_numpy()
for i in range(len(X_test_model)):
  X_test_model[i] = X_test_model[i].lower()
Y = train_df['misogynous']
X_train_model = X[:].to_numpy()
Y_train_model = Y[:].to_numpy()
print(X_train_model.shape, Y_train_model.shape)
print(Y_train_model.sum())

(10000,) (10000,)
5000


In [6]:
MAX_LENGTH = 50
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=MAX_LENGTH):
    features = [] 
    for e in examples:
        input_dict = tokenizer.encode_plus(
            e[3],
            add_special_tokens=True,
            max_length=max_length,
            return_token_type_ids=True,
            pad_to_max_length=True,
            truncation=True,
            return_attention_mask=True,
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e[1]
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

In [9]:
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from transformers import InputExample, InputFeatures
def try_model():
  clf = TFRobertaForSequenceClassification.from_pretrained("roberta-base")
  tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
  print(clf.summary())
  clf.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-6, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  train_bert = pd.DataFrame({
  'guid': range(len(X_train_model)),
  'label': Y_train_model,
  'text_b': ['a']*len(X_train_model),
  'text_a': X_train_model
  })
  bert_train, bert_val = train_test_split(train_bert, test_size = 0.1)
  bert_train, bert_val = bert_train.to_numpy(), bert_val.to_numpy()
  bert_train = convert_examples_to_tf_dataset(bert_train, tokenizer)
  bert_train = bert_train.shuffle(2).batch(64).repeat(2)
  bert_val = convert_examples_to_tf_dataset(bert_val, tokenizer)
  bert_val = bert_val.shuffle(2).batch(64)
  clf.fit(
      bert_train, 
      epochs=5, 
      validation_data=bert_val, verbose = 2, 
      callbacks=[
              tf.keras.callbacks.EarlyStopping(
              monitor="val_accuracy",
              min_delta=0.005,
              patience=4,
              verbose=2,
              mode="max",
              baseline=None,
              restore_best_weights=True,
              ),
              GarbageCollectorCallback()])
  tf_batch = tokenizer(list(X_test_model), max_length=MAX_LENGTH, padding=True, truncation=True, return_tensors='tf')
  tf_outputs = clf(tf_batch)
  label = tf.nn.softmax(tf_outputs[0], axis=-1)
  label = np.array(tf.argmax(label, axis=1))
  return label

In [10]:
from sklearn.metrics import confusion_matrix
Y_test_model = test_labels[1].to_numpy()
Y_train_model = train_df['misogynous'].to_numpy()
y_pred = try_model()
print("######## Misogynous Prediction ########")
print("\nSubmission Score ")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskA = f1_score(Y_test_model, y_pred, average='macro')
subtaskB = f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/627M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 124055040 
 )                                                               
                                                                 
 classifier (TFRobertaClassi  multiple                 592130    
 ficationHead)                                                   
                                                                 
Total params: 124,647,170
Trainable params: 124,647,170
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/5
282/282 - 357s - loss: 0.5786 - accuracy: 0.6814 - val_loss: 0.4720 - val_accuracy: 0.7770 - 357s/epoch - 1s/step
Epoch 2/5
282/282 - 323s - loss: 0.4251 - accuracy: 0.8097 - val_loss: 0.4618 - val_accuracy: 0.7920 - 323s/epoch - 1s/step
Epoch 3/5
282/282 - 324s - loss: 0.3715 - accuracy: 0.8386 - val_loss: 0.4654 - val_accuracy: 0.7980 - 324s/epoch - 1s/step
Epoch 4/5
282/282 - 323s - loss: 0.3248 - accuracy: 0.8648 - val_loss: 0.4888 - val_accuracy: 0.8040 - 323s/epoch - 1s/step
Epoch 5/5
282/282 - 323s - loss: 0.2797 - accuracy: 0.8863 - val_loss: 0.5253 - val_accuracy: 0.8060 - 323s/epoch - 1s/step
######## Misogynous Prediction ########

Submission Score 
[[211 289]
 [ 87 413]] 
 F1 score -  0.6080050375522834


In [11]:
Y_train_model = train_df['shaming'].to_numpy()
Y_test_model = test_labels[2].to_numpy()
y_pred = try_model()
print("\n######## Shaming Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_roberta_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 124055040 
 )                                                               
                                                                 
 classifier (TFRobertaClassi  multiple                 592130    
 ficationHead)                                                   
                                                                 
Total params: 124,647,170
Trainable params: 124,647,170
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/5
282/282 - 352s - loss: 0.3974 - accuracy: 0.8484 - val_loss: 0.3381 - val_accuracy: 0.8760 - 352s/epoch - 1s/step
Epoch 2/5
282/282 - 323s - loss: 0.3144 - accuracy: 0.8758 - val_loss: 0.3307 - val_accuracy: 0.8800 - 323s/epoch - 1s/step
Epoch 3/5
282/282 - 323s - loss: 0.2785 - accuracy: 0.8871 - val_loss: 0.3433 - val_accuracy: 0.8720 - 323s/epoch - 1s/step
Epoch 4/5
282/282 - 323s - loss: 0.2373 - accuracy: 0.9046 - val_loss: 0.3685 - val_accuracy: 0.8640 - 323s/epoch - 1s/step
Epoch 5/5
Restoring model weights from the end of the best epoch: 1.
282/282 - 323s - loss: 0.1978 - accuracy: 0.9231 - val_loss: 0.4046 - val_accuracy: 0.8600 - 323s/epoch - 1s/step
Epoch 00005: early stopping

######## Shaming Prediction ########
[[854   0]
 [146   0]] 
 F1 score -  0.4606256742179072


In [12]:
Y_train_model = train_df['stereotype'].to_numpy()
Y_test_model = test_labels[3].to_numpy()
y_pred = try_model()
print("\n######## Stereotype Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_roberta_for_sequence_classification_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 124055040 
 )                                                               
                                                                 
 classifier (TFRobertaClassi  multiple                 592130    
 ficationHead)                                                   
                                                                 
Total params: 124,647,170
Trainable params: 124,647,170
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/5
282/282 - 352s - loss: 0.5639 - accuracy: 0.7309 - val_loss: 0.4767 - val_accuracy: 0.7770 - 352s/epoch - 1s/step
Epoch 2/5
282/282 - 323s - loss: 0.4510 - accuracy: 0.7944 - val_loss: 0.4659 - val_accuracy: 0.8040 - 323s/epoch - 1s/step
Epoch 3/5
282/282 - 323s - loss: 0.3984 - accuracy: 0.8234 - val_loss: 0.4778 - val_accuracy: 0.7950 - 323s/epoch - 1s/step
Epoch 4/5
282/282 - 323s - loss: 0.3525 - accuracy: 0.8444 - val_loss: 0.5007 - val_accuracy: 0.7990 - 323s/epoch - 1s/step
Epoch 5/5
282/282 - 323s - loss: 0.3070 - accuracy: 0.8658 - val_loss: 0.5479 - val_accuracy: 0.7720 - 323s/epoch - 1s/step

######## Stereotype Prediction ########
[[498 152]
 [185 165]] 
 F1 score -  0.6209697101936653


In [13]:
Y_train_model = train_df['objectification'].to_numpy()
Y_test_model = test_labels[4].to_numpy()
y_pred = try_model()
print("\n######## Objectification Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_roberta_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 124055040 
 )                                                               
                                                                 
 classifier (TFRobertaClassi  multiple                 592130    
 ficationHead)                                                   
                                                                 
Total params: 124,647,170
Trainable params: 124,647,170
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/5
282/282 - 352s - loss: 0.4966 - accuracy: 0.7861 - val_loss: 0.4396 - val_accuracy: 0.8040 - 352s/epoch - 1s/step
Epoch 2/5
282/282 - 324s - loss: 0.3992 - accuracy: 0.8239 - val_loss: 0.4339 - val_accuracy: 0.7980 - 324s/epoch - 1s/step
Epoch 3/5
282/282 - 327s - loss: 0.3512 - accuracy: 0.8474 - val_loss: 0.4323 - val_accuracy: 0.8160 - 327s/epoch - 1s/step
Epoch 4/5
282/282 - 326s - loss: 0.3063 - accuracy: 0.8686 - val_loss: 0.4568 - val_accuracy: 0.8010 - 326s/epoch - 1s/step
Epoch 5/5
282/282 - 327s - loss: 0.2588 - accuracy: 0.8950 - val_loss: 0.4849 - val_accuracy: 0.8050 - 327s/epoch - 1s/step

######## Objectification Prediction ########
[[535 117]
 [177 171]] 
 F1 score -  0.6610966635312344


In [14]:
Y_train_model = train_df['violence'].to_numpy()
Y_test_model = test_labels[5].to_numpy()
y_pred = try_model()
print("\n######## Violence Prediction ########")
print(confusion_matrix(Y_test_model, y_pred),"\n F1 score - ", f1_score(Y_test_model, y_pred, average='macro'))
subtaskB += f1_score(Y_test_model, y_pred, average='macro')*(Y_test_model.sum())

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_roberta_for_sequence_classification_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 124055040 
 )                                                               
                                                                 
 classifier (TFRobertaClassi  multiple                 592130    
 ficationHead)                                                   
                                                                 
Total params: 124,647,170
Trainable params: 124,647,170
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch 1/5
282/282 - 355s - loss: 0.3452 - accuracy: 0.8836 - val_loss: 0.2400 - val_accuracy: 0.9180 - 355s/epoch - 1s/step
Epoch 2/5
282/282 - 327s - loss: 0.2402 - accuracy: 0.9144 - val_loss: 0.2318 - val_accuracy: 0.9200 - 327s/epoch - 1s/step
Epoch 3/5
282/282 - 327s - loss: 0.2028 - accuracy: 0.9303 - val_loss: 0.2347 - val_accuracy: 0.9200 - 327s/epoch - 1s/step
Epoch 4/5
282/282 - 327s - loss: 0.1702 - accuracy: 0.9425 - val_loss: 0.2604 - val_accuracy: 0.9140 - 327s/epoch - 1s/step
Epoch 5/5
Restoring model weights from the end of the best epoch: 1.
282/282 - 326s - loss: 0.1427 - accuracy: 0.9529 - val_loss: 0.2726 - val_accuracy: 0.9080 - 326s/epoch - 1s/step
Epoch 00005: early stopping

######## Violence Prediction ########
[[846   1]
 [141  12]] 
 F1 score -  0.5335759614247613


In [15]:
print("Subtask A score -", 
      subtaskA, 
      "\n Subtask B score - ", 
      subtaskB/(test_labels[5].sum()+ test_labels[4].sum()+ test_labels[3].sum()+test_labels[2].sum()+test_labels[1].sum()))

Subtask A score - 0.6080050375522834 
 Subtask B score -  0.6013974794833647
